In [ ]:
ad = dbutils.secrets.get('Healthcare2.0-SecretScope', 'healthcare-ad')
sp = dbutils.secrets.get('Healthcare2.0-SecretScope', 'Healthcare-sp')
se = dbutils.secrets.get('Healthcare2.0-SecretScope', 'healthcare-secret')

 
spark.conf.set("fs.azure.account.auth.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", sp)
spark.conf.set("fs.azure.account.oauth2.client.secret.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", se)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net", "https://login.microsoftonline.com/" + ad + "/oauth2/token")

In [ ]:
import dlt
input_file = "abfss://sthealthcare2@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/pbiPatientPredictiveSetv4.csv"
@dlt.table()
def patient_predictive_bronze():
  return (spark.read.format("csv").option("header",True).load(input_file))

@dlt.table()
def patient_predictive_silver():
  return (dlt.read("patient_predictive_bronze"))

@dlt.table()
def patient_predictive_gold():
  return (dlt.read("patient_predictive_silver"))